In [1]:
import os
import os.path
import numpy as np
import random
import torch
import json
import torch.utils.data as data
from torchvision import transforms
from PIL import Image
import pickle

/l/vision/zekrom_ssd/fraramir/miniconda3/envs/torch-gpu/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# print(x.shape)
# print(y.shape)
# train_ego_idx = np.array(train_video_ego_id) == 1
# train_exo_idx = np.array(train_video_ego_id) == 0
# val_ego_idx = np.array(val_video_ego_id) == 1
# val_exo_idx = np.array(val_video_ego_id) == 0
# # (20642, 128)
# (20642,)
shapes = 0
for i in range(1984, 2141):
    try:
        x = np.load(f"/nfs/wattrel/data/md0/datasets/AE2/AE2_data/tennis_forehand/exo/{i}_cf_epoch7.npy")
        shapes += x.shape[-1]
    except FileNotFoundError:
        continue
# print(shapes)

# shapes = 0
for i in os.listdir("/nfs/wattrel/data/md0/datasets/AE2/AE2_data/tennis_forehand/ego"):
    if i.endswith("_cf_epoch7.npy"):
        x = np.load(f"/nfs/wattrel/data/md0/datasets/AE2/AE2_data/tennis_forehand/ego/{i}")
        video_name = i.replace('_cf_epoch7.npy', '').split('/')[-1]
        labels = dircy[video_name]
        assert x.shape[-1] == len(labels)
        shapes += x.shape[-1]
print(shapes)

9484


In [2]:
def _construct_video_path_by_mode(dir_name, mode):
    video_paths = []
    f_out = open(os.path.join(dir_name, mode + '.csv'), 'r')
    for line in f_out:
        line = line.strip()
        base_name = os.path.splitext(line)[0]  # Remove .mp4
        new_name = base_name + '_cf_epoch7.npy'
        video_paths.append(os.path.join(dir_name, new_name))
    return video_paths

In [3]:
vids0 = _construct_video_path_by_mode("/nfs/wattrel/data/md0/datasets/AE2/AE2_data/break_eggs/ego", "test")
vids1 = _construct_video_path_by_mode("/nfs/wattrel/data/md0/datasets/AE2/AE2_data/break_eggs/exo", "test")

In [ ]:
with open(os.path.join("/nfs/wattrel/data/md0/datasets/AE2/AE2_data/break_eggs", 'label.pickle'), 'rb') as handle:
            dircy = pickle.load(handle)
feats = []
labels = []
for i in vids0:
    x = np.load(i)
    video_name = i.replace('_cf_epoch7.npy', '').split('/')[-1]
    label = dircy[video_name]
    # print(torch.from_numpy(x).shape)
    feats.append(x)
    labels.append(label)

# for i in vids1:
#     x = np.load(i)
#     video_name = i.replace('_cf_epoch7.npy', '').split('/')[-1]
#     label = dircy[video_name]
#     # print(torch.from_numpy(x).shape)
#     feats.append(x)
#     labels.append(label)

In [5]:

feats = np.concatenate(feats, axis=-1)
# labels = np.stack(labels)
print(feats.shape)
# print(labels.shape) # 3105

(768, 3758)


In [6]:
labels = np.concatenate(labels)

In [7]:
print(labels.shape)

(3758,)


In [8]:
feats0 = feats.T.copy()
print(feats0.shape)

(3758, 768)


In [ ]:
# np.save("/nfs/wattrel/data/md0/datasets/AE2/AE2_data/embeddings/break_eggs/test_embeds.npy", feats0)
# np.save("/nfs/wattrel/data/md0/datasets/AE2/AE2_data/embeddings/break_eggs/test_label.npy", labels)

In [29]:
class GTEA_FRAMES():
    def __init__(self,
                 root='/nfs/wattrel/data/md0/datasets/action_seg_datasets/GTEA',
                 small_test=False,
                 frame_dir='/nfs/wattrel/data/md0/datasets/action_seg_datasets/GTEA/frames/',
                 save_feat_dir='gtea_vit_features',
                 transform=None,
                 sliding_window=15):
        self.root = root
        self.small_test = small_test
        self.frame_dir = frame_dir
        self.save_feat_dir = save_feat_dir
        self.transform = transform
        self.sliding_window = sliding_window
        all_files = os.walk(self.frame_dir)
        self.convert_tensor = transforms.ToTensor()
        self.data_lst = []
        for path, dir, filelst in all_files:
            if len(filelst) > 0:
                self.data_lst.append((filelst, path))

    def get_it(self, index):
        # window_size = self.sliding_window
        videoname = self.data_lst[index]
        vroot = videoname[1]
        path_list = videoname[0]
        # vlen = len(path_list)
        path_list.sort(key=lambda x: int(x[4:-4]))
        # seq = [Image.open(os.path.join(vroot, p)).convert('RGB') for p in path_list]

        # if self.sliding_window:
        #     first_frame = seq[0]
        #     last_frame = seq[-1]
        #     seq = [first_frame] * ((self.sliding_window-1)//2) + seq + [last_frame] * ((self.sliding_window-1)//2)

        # if self.transform is not None:
        #     seq = self.transform(seq)
        # else:
        #     convert_tensor = transforms.ToTensor()
        #     seq = [convert_tensor(img) for img in seq]
        #     seq = torch.stack(seq)
        vsplt = vroot.split('/')[-1]
        fname = vsplt + '.npy'

In [30]:
xx = GTEA_FRAMES()

In [32]:
print(xx.data_lst)# xx.get_it(0)

[]


In [11]:
root='/nfs/wattrel/data/md0/kung/state-aware-video-pretrain/data/breakfast'
data_lst = np.load(
            os.path.join(root, 'splits', 'breakfast_exfm.npy'))
frame_dir='/nfs/wattrel/data/md0/datasets/action_seg_datasets/breakfast/frames/'
num_frames= 15
def frame_sampler(videoname, vlen):
        start_idx = int(videoname[1])
        seq_idx = np.arange(num_frames) + start_idx
        print("seq_idx ", seq_idx)
        seq_idx = np.where(seq_idx < vlen, seq_idx, vlen - 1)
        return seq_idx

def get_it(index):
    videoname = data_lst[index]
    print("videoname ", videoname)
    vroot = videoname[0]
    vsplt = vroot.split('_', 2)
    print("vroot ", vroot)
    print("vsplt ", vsplt)

    vname_splt = np.copy(vsplt)
    if vsplt[1] == 'stereo':
        vname_splt[1] = 'stereo01'
        vname_splt[2] = vsplt[2][:-4]
    vpath = os.path.join(frame_dir, *vsplt)
    vlen = len([f for f in os.listdir(vpath) if os.path.isfile(os.path.join(vpath, f))])
    path_list = os.listdir(vpath)
    print("vpath ", vpath)
    print("vlen ", vlen)
    path_list.sort(key=lambda x: int(x[4:-4]))
    frame_index = frame_sampler(videoname, vlen)
    print("frame_index ", frame_index)
    seq = [Image.open(os.path.join(vpath, path_list[i])).convert('RGB') for i in frame_index]
    print("seq ", len(seq))
    if 1:
        convert_tensor = transforms.ToTensor()
        seq = [convert_tensor(img) for img in seq]
        seq = torch.stack(seq)
    fname = vname_splt[0] + '_' + vname_splt[1] + '_' + vname_splt[2] + '_' + videoname[1] + '.npy'
    print("fname ", fname)
    return seq, fname

In [12]:
x,y = get_it(120)

videoname  ['P03_cam01_P03_friedegg' '2048']
vroot  P03_cam01_P03_friedegg
vsplt  ['P03', 'cam01', 'P03_friedegg']
vpath  /nfs/wattrel/data/md0/datasets/action_seg_datasets/breakfast/frames/P03/cam01/P03_friedegg
vlen  4268
seq_idx  [2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060 2061
 2062]
frame_index  [2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060 2061
 2062]
seq  15
fname  P03_cam01_P03_friedegg_2048.npy


In [ ]:
video_name = video.replace('_cf_epoch7.npy', '').split('/')[-1]
view = video.split('/')[-2]
labels = self.label_dict[video_name]
# print(np.load(video).shape,video_frames_count, len(labels))
            # continue

In [10]:
import pickle
with open(os.path.join("/nfs/wattrel/data/md0/datasets/AE2/AE2_data/tennis_forehand", 'label.pickle'), 'rb') as handle:
            dircy = pickle.load(handle)
print(dircy)

{'2037': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), '2036': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), '2008': array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), '2020': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1]), '2034': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), '2035': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1]), '2025': array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]), '2019': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1]), '2018': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1]), '1987': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1]), '2040': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1

In [4]:
def find_frames_folders(parent_dir):
    """Recursively find all folders ending with '_frames' and print their paths."""
    for root, dirs, files in os.walk(parent_dir):
        for d in dirs:
            if d.endswith("_frames"):
                full_path = os.path.join(root, d)
                print(full_path)

def percent_folders_with_few_images(parent_dir, threshold):
    """Print the percentage of '_frames' folders that have less than `threshold` images."""
    total_folders = 0
    folders_below_threshold = 0
    all_paths = []
    for root, dirs, files in os.walk(parent_dir):
        for d in dirs:
            if d.endswith("_frames"):
                total_folders += 1
                full_path = os.path.join(root, d)
                
                image_count = sum(
                    1 for f in os.listdir(full_path)
                    if f.lower().endswith(('.jpg'))
                )

                if image_count < threshold:
                    folders_below_threshold += 1
                
                all_paths.append([str(full_path), str(image_count)])
                # else:
                # for j in range(0,image_count, 16):
                #     all_paths.append([str(full_path), str(j), str(image_count)])

    if total_folders == 0:
        print("No '_frames' folders found.")
    else:
        percentage = (folders_below_threshold / total_folders) * 100
        print(f"{folders_below_threshold} out of {total_folders} folders have less than {threshold} images.")
        print(f"That's {percentage:.2f}%.")
    
    return all_paths

In [2]:
import numpy as np
x = np.load("/nfs/wattrel/data/md0/datasets/AE2/AE2_data/break_eggs/ego/S31_75.243_86.4_cf_epoch7.npy")
print(x.shape)

(768, 334)


In [10]:
import cv2
import os
cap = cv2.VideoCapture("/nfs/wattrel/data/md0/datasets/AE2/AE2_data/break_eggs/ego/S37_90.93_98.388.mp4")
    # Find the number of frames
video_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) #-1

image_count = sum(
                    1 for f in os.listdir("/nfs/wattrel/data/md0/datasets/AE2/AE2_data/break_eggs/ego/S37_90.93_98.388" + '_frames')
                    if f.lower().endswith(('.jpg'))
                )

print(video_length, image_count)

224 223


In [5]:

parent_directory = "/nfs/wattrel/data/md0/datasets/AE2/AE2_data"
# for root, dirs, files in os.walk(parent_directory):
#     print(root)
#     print(dirs)
#     print(files)
#     print('-'*20)
# find_frames_folders(parent_directory)
paths = percent_folders_with_few_images(parent_directory, 16)

np.save("/nfs/wattrel/data/md0/datasets/AE2/AE2_metadata", np.array(paths), allow_pickle=True)

9 out of 782 folders have less than 16 images.
That's 1.15%.


In [13]:
data_lst = np.load(
            "/nfs/wattrel/data/md0/datasets/AE2/AE2_metadata.npy")
# print(os.listdir(data_lst[0][0]))
print(data_lst[0][0][:-7] + '.npy')
print(data_lst[0][0].split('/')[-1])

/nfs/wattrel/data/md0/datasets/AE2/AE2_data/pour_milk/test/ego/subject3_o1_1_cam4.npy
subject3_o1_1_cam4_frames


In [7]:
x = "img_00018.jpg"
print(x[4:-4])

00018


In [16]:
root='/nfs/wattrel/data/md0/kung/state-aware-video-pretrain/data/breakfast'
data_lst = np.load(
            os.path.join(root, 'splits', 'breakfast_exfm.npy'))
print(data_lst)

[['P03_cam01_P03_cereals' '0']
 ['P03_cam01_P03_cereals' '32']
 ['P03_cam01_P03_cereals' '64']
 ...
 ['P54_webcam02_P54_tea' '896']
 ['P54_webcam02_P54_tea' '928']
 ['P54_webcam02_P54_tea' '960']]
